In [1]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [2]:
import numpy as np
from interfacegan_utils import calculate_boundary
from glob import glob
import pandas as pd
from tqdm import tqdm

In [3]:
latent_input_base_path = f"{DATA_PATH}/Models/InterfaceGAN/Inputs/e4e_00005/"
all_paths = glob(f"{latent_input_base_path}/*/*")
all_paths = [path for path in all_paths if not "summary_stats" in path]

In [4]:
for path in tqdm(all_paths):
    out_dir = path.replace('Inputs', 'Outputs')
    if os.path.exists(out_dir):
        continue

    else:
        accuracies = []
        os.makedirs(out_dir, exist_ok=True)
        for dim in range(16):
            latent_codes_path = f"{path}/latents_dim_{dim}.npy"
            scores_path = f"{path}/target.npy"
            out_name = f'boundary_dim{dim}.npy'
            acc = calculate_boundary(out_dir, out_name, latent_codes_path, scores_path)
            accuracies.append(acc)

        # save result
        attribute = path.split('/')[-2]
        values = path.split('/')[-1].split('_')

        # Crate dataframe with columns 'attribute', 'value1, 'value2', 'accuracy1', 'accuracy2', ...
        result = pd.DataFrame(columns=['attribute'] + ['neg_class', 'pos_class'] + [f'acc_dim_{i}' for i in range(len(accuracies))])
        result.loc[0] = [attribute] + values + accuracies
        result.to_csv(f"{out_dir}/summary_stats.csv", index=False)

100%|██████████| 884/884 [02:07<00:00,  6.92it/s]  


In [6]:
summary_paths = glob(f"{latent_input_base_path.replace('Inputs', 'Outputs')}/*/*/summary_stats.csv")

training_results = pd.concat([pd.read_csv(path) for path in summary_paths])
training_results

,attribute,neg_class,pos_class,acc_dim_0,acc_dim_1,acc_dim_2,acc_dim_3,acc_dim_4,acc_dim_5,acc_dim_6,acc_dim_7,acc_dim_8,acc_dim_9,acc_dim_10,acc_dim_11,acc_dim_12,acc_dim_13,acc_dim_14,acc_dim_15
0,sleeve_length,Sleeveless,Elbow length,0.993028,0.995020,0.995020,0.996016,0.997012,0.990040,0.998008,0.996016,0.997012,0.992032,0.995020,0.996016,0.992032,0.994024,0.996016,0.996016
0,sleeve_length,Extra short,Elbow length,0.984055,0.986333,0.984055,0.995444,0.993166,0.984055,0.997722,0.984055,0.981777,0.990888,0.988610,0.981777,0.990888,0.995444,0.993166,0.984055
0,sleeve_length,34 length,Mixed,0.957831,0.969880,0.977912,0.959839,0.961847,0.971888,0.963855,0.949799,0.969880,0.945783,0.959839,0.963855,0.963855,0.975904,0.971888,0.967871
0,sleeve_length,Mixed,Extra long,0.941441,0.959459,0.954955,0.936937,0.954955,0.941441,0.959459,0.923423,0.905405,0.914414,0.918919,0.900901,0.932432,0.945946,0.932432,0.936937
0,sleeve_length,Extra short,Strapless,0.944649,0.952030,0.963100,0.974170,0.970480,0.970480,0.985240,0.974170,0.944649,0.959410,0.963100,0.937269,0.966790,0.944649,0.948339,0.959410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,price,threshold,100,0.688241,0.685870,0.687293,0.688952,0.696064,0.701517,0.688952,0.685870,0.693457,0.698435,0.693694,0.695827,0.691560,0.690375,0.707681,0.690138
0,price,threshold,500,0.964675,0.961830,0.964438,0.964438,0.963016,0.958511,0.964675,0.963016,0.963964,0.962779,0.962541,0.966572,0.962541,0.962067,0.963490,0.967994
0,price,threshold,400,0.948791,0.945235,0.947368,0.946894,0.941204,0.943338,0.945946,0.948080,0.946657,0.951873,0.950450,0.947843,0.947843,0.947131,0.945472,0.949028
0,price,threshold,300,0.906354,0.902798,0.906591,0.907539,0.903983,0.907539,0.914414,0.912755,0.905168,0.911095,0.907065,0.913940,0.910384,0.908013,0.912281,0.901138


In [10]:
training_results[training_results.pos_class == 'Kraimod'].sort_values('acc_dim_0', ascending=False)

,attribute,neg_class,pos_class,acc_dim_0,acc_dim_1,acc_dim_2,acc_dim_3,acc_dim_4,acc_dim_5,acc_dim_6,acc_dim_7,acc_dim_8,acc_dim_9,acc_dim_10,acc_dim_11,acc_dim_12,acc_dim_13,acc_dim_14,acc_dim_15
0,brand,Lauren Ralph Lauren,Kraimod,1.000000,1.000000,1.000000,1.000000,0.989583,1.000000,1.000000,1.000000,0.989583,1.000000,1.000000,1.000000,0.989583,1.000000,1.000000,1.000000
0,brand,b.young,Kraimod,0.990099,0.940594,0.950495,0.970297,0.960396,0.970297,0.960396,0.990099,0.960396,0.960396,0.940594,0.960396,0.940594,0.940594,0.891089,0.940594
0,brand,Guess,Kraimod,0.989474,0.947368,0.936842,0.968421,0.894737,0.957895,0.957895,0.957895,1.000000,0.936842,0.926316,0.894737,0.936842,0.978947,0.947368,0.947368
0,brand,Sheego,Kraimod,0.989247,0.978495,0.967742,0.946237,0.989247,0.989247,0.978495,0.989247,0.989247,1.000000,0.978495,0.978495,0.956989,1.000000,1.000000,0.967742
0,brand,Bershka,Kraimod,0.988095,1.000000,1.000000,0.976190,0.988095,0.976190,0.988095,1.000000,0.988095,1.000000,0.988095,0.976190,1.000000,1.000000,0.988095,1.000000
0,brand,Culture,Kraimod,0.987654,1.000000,0.987654,1.000000,0.975309,1.000000,1.000000,1.000000,0.987654,0.975309,0.987654,0.950617,0.975309,0.975309,0.962963,0.975309
0,brand,Even&Odd,Kraimod,0.987500,0.925000,0.925000,0.987500,0.975000,1.000000,0.987500,0.962500,0.962500,0.950000,0.962500,0.962500,0.987500,0.987500,0.962500,0.975000
0,brand,ONLY,Kraimod,0.981818,0.963636,0.972727,0.963636,0.963636,0.981818,0.909091,0.936364,0.972727,0.990909,0.927273,0.972727,0.954545,0.963636,0.954545,0.954545
0,brand,Next,Kraimod,0.978947,0.994737,0.973684,0.989474,0.968421,0.984211,0.978947,0.994737,0.994737,0.984211,0.973684,0.957895,1.000000,0.973684,0.984211,0.989474
0,brand,Anna Field,Kraimod,0.978495,0.989247,0.967742,0.989247,0.967742,0.956989,0.935484,0.967742,0.946237,0.989247,0.956989,0.956989,0.978495,0.978495,0.978495,0.967742
